In [14]:
import pandas as pd
import os
import re
from collections import OrderedDict
from sqlalchemy import create_engine, MetaData, Table
import inflect
import datetime

In [15]:
cricket = create_engine('sqlite:///cricket.sql', echo=False)

In [17]:
Matches = OrderedDict()
Batsmen = []
Bowlers = []
Players = []

directory = os.path.join("../data")
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith("ing.csv"):
            df = pd.read_csv("../data/" + file)
            match_no = int(re.search(r'\d+', file).group())
            if match_no not in Matches.keys():
                Matches[match_no] = OrderedDict()
            if "bowling" in file:
                
                for bowler in df['Bowling']:
                    if bowler not in Bowlers:
                        Bowlers.append(bowler)
                    if bowler not in Players:
                        Players.append(bowler)
                        
                Matches[match_no]["bowling"] = OrderedDict()
                Matches[match_no]["bowling"]["data"] = df
                Matches[match_no]["bowling"]["index"] = df.loc[df['Bowling'] == 'Bowling'].index.tolist()[0]
                
            else:
#                 df[pd.to_numeric]
#                 print(df[pd.to_numeric(df.R, errors='coerce').isnull()])
#                 print(df['R'])
#                 print(df['R'].iloc[8])
#                 print(df[:df.loc[df['BATSMEN'] == 'BATSMEN'].index.tolist()[0]])
#                 print()
#                 df['R'] = df['R'].apply(pd.to_numeric)
#                 pd.to_numeric(df['R'])
                for batsman in df['BATSMEN']:
                    if batsman not in Batsmen:
                        Batsmen.append(batsman)
                    if batsman not in Players:
                        Players.append(batsman)
                Matches[match_no]["batting"] = OrderedDict()
                Matches[match_no]["batting"]["index"] = df.loc[df['BATSMEN'] == 'BATSMEN'].index.tolist()[0]
                df.rename(columns={'Unnamed: 1':'FOW'}, inplace=True)
                Matches[match_no]["batting"]["data"] = df
                

In [18]:
# First innings batting details of the first match
Matches[1]["batting"]["data"][:Matches[1]["batting"]["index"]]

,BATSMEN,FOW,R,B,4s,6s,SR
0,RG Sharma (c),c Rayudu b Watson,15,18,1,1,83.33
1,E Lewis,lbw b Chahar,0,2,0,0,0.00
2,Ishan Kishan †,c Wood b Imran Tahir,40,29,4,1,137.93
3,SA Yadav,c Harbhajan Singh b Watson,43,29,6,1,148.27
4,HH Pandya,not out,22,20,2,0,110.00
5,KH Pandya,not out,41,22,5,2,186.36


In [19]:
# Second innings batting details of the first match
Matches[1]["batting"]["data"][Matches[1]["batting"]["index"] + 1:]

,BATSMEN,FOW,R,B,4s,6s,SR
7,SR Watson,c Lewis b HH Pandya,16,14,1,1,114.28
8,AT Rayudu,lbw b Markande,22,19,4,0,115.78
9,SK Raina,c KH Pandya b HH Pandya,4,6,0,0,66.66
10,KM Jadhav,not out,24,22,1,2,109.09
11,MS Dhoni (c) †,lbw b Markande,5,5,0,0,100.00
12,RA Jadeja,c Yadav b Mustafizur Rahman,12,13,1,0,92.30
13,DJ Bravo,c Sharma b Bumrah,68,30,3,7,226.66
14,DL Chahar,st †Ishan Kishan b Markande,0,1,0,0,0.00
15,Harbhajan Singh,c Bumrah b McClenaghan,8,5,1,0,160.00
16,MA Wood,c Mustafizur Rahman b HH Pandya,1,3,0,0,33.33


In [20]:
Matches[1]["bowling"]["data"][:Matches[1]["bowling"]["index"]]

,Bowling,O,M,R,W,Econ,0s,4s,6s,WD,NB
0,DL Chahar,3,0,14,1,4.66,12,2,0,0,0
1,SR Watson,4,0,29,2,7.25,12,1,2,1,0
2,Harbhajan Singh,2,0,14,0,7.00,3,1,0,0,0
3,RA Jadeja,1,0,9,0,9.00,3,2,0,0,0
4,MA Wood,4,0,49,0,12.25,6,6,2,1,0
5,Imran Tahir,2,0,23,1,11.50,4,3,1,0,0
6,DJ Bravo,4,0,25,0,6.25,9,3,0,0,0


In [21]:
Matches[1]["bowling"]["data"][Matches[1]["bowling"]["index"] + 1:]

,Bowling,O,M,R,W,Econ,0s,4s,6s,WD,NB
8,MJ McClenaghan,4,0,44,1,11.00,5,4,2,1,0
9,Mustafizur Rahman,3.5,0,39,1,10.17,8,6,1,0,0
10,JJ Bumrah,4,0,37,1,9.25,10,0,4,0,1
11,HH Pandya,4,0,24,3,6.00,12,1,1,3,0
12,M Markande,4,0,23,3,5.75,11,0,2,0,0


In [32]:
# Other Match Data
match_data = pd.read_csv("../data/match-data.csv")
# Change the date format to mm/dd/yy
for index, row in match_data.iterrows():
    row['Match Date'] = datetime.datetime.strptime(row['Match Date'], '%b %d, %Y').strftime('%m/%d/%y')
match_data

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard
0,Mum Indians,Super Kings,Super Kings,1 wicket,Mumbai,04/07/18,T20
1,Kings XI,Daredevils,Kings XI,6 wickets,Mohali,04/08/18,T20
2,KKR,RCB,KKR,4 wickets,Kolkata,04/08/18,T20
3,Sunrisers,Royals,Sunrisers,9 wickets,Hyderabad (Deccan),04/09/18,T20
4,Super Kings,KKR,Super Kings,5 wickets,Chennai,04/10/18,T20
5,Royals,Daredevils,Royals,10 runs,Jaipur,04/11/18,T20
6,Sunrisers,Mum Indians,Sunrisers,1 wicket,Hyderabad (Deccan),04/12/18,T20
7,RCB,Kings XI,RCB,4 wickets,Bengaluru,04/13/18,T20
8,Mum Indians,Daredevils,Daredevils,7 wickets,Mumbai,04/14/18,T20
9,KKR,Sunrisers,Sunrisers,5 wickets,Kolkata,04/14/18,T20


In [80]:
teams = []
match_dates = [] 
grounds = []
# Team names and match_dates
for t,d in zip(match_data['Team 1'], match_data['Match Date']):
    if t not in teams:
        teams.append(t)
    
    if d not in match_dates:
        match_dates.append(d)

# Ground locations for matches
for g in match_data['Ground']:
    if g not in grounds:
        grounds.append(g)

In [128]:
p = inflect.engine()
for k1,v1 in Matches.items():
#     k1 - match no
    for k2, v2 in v1.items():
        # k2 - batting,bowling
        word = p.number_to_words(k1, group=1)
        numbers = word.split(",")
        string = ''
#         print(numbers)
        for num in numbers:
            string += num.lstrip() + "_"
        string += k2 + "_"
        index = v2['index']
        i1 = v2['data'][:v2['index']]
#         i1['R'] = i1['R'].apply(pd.to_numeric)
#         print(i1.dtypes)
        i2 = v2['data'][v2['index'] + 1:]
        i1.to_sql(name = string + p.number_to_words(1), con = cricket, if_exists='replace')
        i2.to_sql(name = string + p.number_to_words(2), con = cricket, if_exists='replace')


In [124]:
cricket.execute("SELECT * from two_five_batting_two").fetchall()

[(8, 'KL Rahul †', ' b Rashid Khan', '32', '26', '4', '1', '123.07'),
 (9, 'CH Gayle', 'c & b Basil Thampi', '23', '22', '1', '2', '104.54'),
 (10, 'MA Agarwal', 'c Pandey b Shakib Al Hasan', '12', '15', '0', '0', '80.00'),
 (11, 'KK Nair', 'lbw b Rashid Khan', '13', '17', '0', '0', '76.47'),
 (12, 'AJ Finch', 'c Pandey b Shakib Al Hasan', '8', '4', '0', '1', '200.00'),
 (13, 'MK Tiwary', 'c Williamson b Sandeep Sharma', '1', '5', '0', '0', '20.00'),
 (14, 'R Ashwin (c)', 'c Williamson b Rashid Khan', '4', '6', '0', '0', '66.66'),
 (15, 'AJ Tye', 'lbw b Sandeep Sharma', '4', '2', '1', '0', '200.00'),
 (16, 'BB Sran', 'run out (†Saha)', '2', '5', '0', '0', '40.00'),
 (17, 'AS Rajpoot', ' b Basil Thampi', '8', '9', '1', '0', '88.88'),
 (18, 'Mujeeb Ur Rahman', 'not out', '10', '5', '2', '0', '200.00')]

In [ ]:
## Questions queried only from match-data.csv

ques = "Who was the winner between Mum Indians and Super Kings on Apr 7, 2018?"
# query:<who> <Team1> <Team2> < Match Date>, select col: <Winner>
ques = "Who was the winner between Kings XI and Daredevils at Mohali?"
# query:<who> <Team1> <Team2> <Ground>, select col: <Winner>
ques = "Where was the match between Sunrisers and Royals held on Apr 9, 2018?"
# query: <where> <Team1> <Team2> <match date>, select col: <Ground>
ques = "When was the match between Super Kings and KKR held in Chennai?"
# query: <when> <team1> <team2> <ground>, select col: <match date>

In [ ]:
# Need to link match-data.csv and scoreboards for individual matches
# For questions such as : How much did <Player> score in match between <team1> and <team2>?
## first query team1 & team2 in match-data.csv, find index of row then go to scoreboard-match-data-<index>.csv for
## that matches score